# KNN - Model

In [7]:
import pandas as pd
import numpy as np
import pickle
import time
import os
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.neighbors import NearestNeighbors


class SpotifyRecommender:
    supported = ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence'] + [f'Chroma_{i}' for i in range(1, 13)] + [f'MEL_{i}' for i in range(1, 129)] + [f'MFCC_{i}' for i in range(1, 49)] + [f'Spectral_contrast_{i}' for i in range(1, 8)] + [f'Tonnetz_{i}' for i in range(1, 7)] + ['ZCR', 'entropy_energy', 'spectral_bandwith', 'spectral_centroid', 'spectral_rollOff_max', 'spectral_rollOff_min']
    return_columns = ['name','artists_name','artists_genres','album_name','track_href','preview_url','analysis_url','href','lyrics','playlist','popularity','tempo','time_signature','track_id',
             'artists_followers', 'artists_artist_popularity', 'artists_id','album_release_date','album_images','album_total_tracks','album_external_urls','album_id']
    def __init__(self,load_model="",k=300):
        self.scaler = None
        self.pca = None
        self.sfm = None
        self.knn = None
        self.k = k
        self.filepath = 'saved_modelV2.pkl'
        self.dataset = None
        self.trained_features = []
        if not load_model == "":
            path = os.path.join(load_model)
            self.load(path)
        

    def __get_features(self,data,features):
        listed = data.columns.tolist()
        for f in listed:
            if f in self.supported and f in features:
                self.trained_features.append(f)
        return
    
    def __match_genre(self, row, target_list):
        target_words = set(word for genre in target_list for word in genre.split(' '))
        row_words = set(word for genre in row for word in genre.split(' '))
        return len(row_words.intersection(target_words))


    def train(self,dataset,n_lines=0,features=supported):
        self.__get_features(dataset,features)
        self.dataset = dataset

        if not n_lines == 0:
            dataset = dataset.sample(n=n_lines)

        self.scaler = StandardScaler()
        scaled_data = self.scaler.fit_transform(dataset[self.trained_features])

        rf = RandomForestRegressor(n_estimators=100, random_state=42)
        rf.fit(scaled_data, dataset.index)  # Hier verwende ich data.index anstelle von target

        self.sfm = SelectFromModel(rf, threshold='mean')
        self.sfm.fit(scaled_data, dataset.index)  # Hier verwende ich data.index anstelle von target

        self.pca = PCA(n_components=0.95)
        reduced_data = self.pca.fit_transform(scaled_data[:, self.sfm.get_support()])

        self.knn = NearestNeighbors(n_neighbors=self.k)
        self.knn.fit(reduced_data)

    def predict(self, df_selected,k=None,match_genre=True):
        if not k==None:
            self.knn.n_neighbors = k

        new_scaled_data = self.scaler.transform(df_selected[self.trained_features])
        new_reduced_data = self.pca.transform(new_scaled_data[:, self.sfm.get_support()])
        distances, indices = self.knn.kneighbors(new_reduced_data)
        
        self.knn.n_neighbors = self.k
        result = self.dataset[self.return_columns].iloc[indices[0]]
        result['distances'] = distances[0]

        result = result.sort_values('album_release_date', ascending=False)
        result = result.drop_duplicates(subset=['name', 'artists_name'], keep='first')

        if match_genre:
            result['match_genre_count'] = result['artists_genres'].apply(self.__match_genre, target_list=df_selected['artists_genres'].tolist()[0])
            return result.sort_values(by=['match_genre_count','distances'], ascending=[False,True])
        else:
            return result.sort_values(by='distances',ascending=True)

    def save(self, file_path=None):
        if file_path == None:
            file_path = self.filepath
        model_data = {
            'knn': self.knn,
            'scaler': self.scaler,
            'pca': self.pca,
            'sfm': self.sfm,
            'dataset': self.dataset,
            'filepath': file_path,
            'trained_features': self.trained_features
        }
        with open(file_path, 'wb') as f:
            pickle.dump(model_data, f)

    def load(self, file_path):
        with open(file_path, 'rb') as f:
            model_data = pickle.load(f)

        self.knn = model_data['knn']
        self.scaler = model_data['scaler']
        self.pca = model_data['pca']
        self.sfm = model_data['sfm']
        self.filepath = model_data['filepath']
        self.trained_features =  model_data['trained_features']
        self.dataset = model_data['dataset']

# ETL - Prozess

In [8]:
import os
import pandas as pd
import ast

# @op
def str_to_list(s):
    s = str(s)[1:-1]
    if len(s) == 0:
        s = "'not defined'"
    items = s.split(',')
    return [item.replace("'","") for item in items]

# @asset
def import_albums():
    albums_path = os.path.join('spotify_data/Data Sources/spotify_albums.csv')
    return pd.read_csv(albums_path,sep=',',index_col='Unnamed: 0').rename(columns=lambda x: "album_" + x)

# @asset
def import_artists():
    albums_path = os.path.join('spotify_data/Data Sources/spotify_artists.csv')
    return pd.read_csv(albums_path,sep=',',index_col='Unnamed: 0').rename(columns=lambda x: "artists_" + x)

# @asset
def import_tracks():
    albums_path = os.path.join('spotify_data/Data Sources/spotify_tracks.csv')
    return pd.read_csv(albums_path,sep=',',index_col='Unnamed: 0').rename(columns={'id':'track_id','artists_id':'track_artists_id'})

# @asset
def import_lyrics_features():
    albums_path = os.path.join('spotify_data/Features Extracted/lyrics_features.csv')
    df = pd.read_csv(albums_path,sep=',',index_col='Unnamed: 0')
    return df

# @asset
def import_audio_features():
    albums_path = os.path.join('spotify_data/Features Extracted/low_level_audio_features.csv')
    df = pd.read_csv(albums_path,sep=',',index_col='Unnamed: 0')
    return df

# @op
def transform_albums(albums_raw):
    df = albums_raw.copy()
    df['album_available_markets'] = df['album_available_markets'].apply(lambda x: str_to_list(x))
    df['album_release_date'] = pd.to_datetime(df['album_release_date'])
    df['album_external_urls'] = df['album_external_urls'].apply(ast.literal_eval)
    df['album_images'] = df['album_images'].apply(ast.literal_eval)
    
    return df

# @op
def transform_artists(artists_raw):
    df = artists_raw.copy()
    df['artists_genres'] = df['artists_genres'].apply(lambda x: str_to_list(x))

    return df

# @op
def transform_tracks(tracks_raw):
    df = tracks_raw.copy()
    df['available_markets'] = df['available_markets'].apply(lambda x: str_to_list(x))
    df['track_artists_id'] = df['track_artists_id'].apply(lambda x: str_to_list(x))

    return df

# @op
def match_spotify_data(tracks,albums,artists,audio_features,lyrics_features):
    tracks = tracks.explode('track_artists_id') # tack zeile pro artist

    merged = pd.merge(tracks, albums, left_on='album_id', right_on='album_id', how='inner')
    tracks_albums_artists = pd.merge(merged, artists, left_on='track_artists_id', right_on='artists_id', how='inner')
    # inner -> 101939 rows × 53 columns

    tracks_albums_artists_audio = pd.merge(tracks_albums_artists, audio_features, left_on='track_id', right_on='track_id', how='inner')
    tracks_albums_artists_audio_lyrics = pd.merge(tracks_albums_artists_audio, lyrics_features, left_on='track_id', right_on='track_id', how='inner')
    # inner -> 94924 rows × 266 columns

    return tracks_albums_artists_audio_lyrics

# @op
def train_model(df,n_lines,features,filename):
    model = SpotifyRecommender()
    if features == None:
        model.train(df, n_lines)
    else:
        model.train(df, n_lines, features)
    model.save(filename)

# @job
def create_full_model():
    raw_albums = import_albums()
    raw_artists = import_artists()
    raw_tracks = import_tracks()
    raw_lyrics_features = import_lyrics_features()
    raw_audio_features = import_audio_features()

    albums = transform_albums(raw_albums)
    artists = transform_artists(raw_artists)
    tracks = transform_tracks(raw_tracks)
    lyrics_features = raw_lyrics_features
    audio_features = raw_audio_features
    
    df = match_spotify_data(tracks,albums,artists,audio_features,lyrics_features)
    
    train_model(df,0,None,"SpotifyRecommender_allFeatures.pkl")


# @job
def create_apiFeatures_model():
    features = ['acousticness', 'danceability', 'duration_ms', 
                'energy', 'instrumentalness', 'key', 'liveness', 
                'loudness', 'mode', 'speechiness', 'tempo', 
                'time_signature','valence']
    raw_albums = import_albums()
    raw_artists = import_artists()
    raw_tracks = import_tracks()
    raw_lyrics_features = import_lyrics_features()
    raw_audio_features = import_audio_features()

    albums = transform_albums(raw_albums)
    artists = transform_artists(raw_artists)
    tracks = transform_tracks(raw_tracks)
    lyrics_features = raw_lyrics_features
    audio_features = raw_audio_features
    
    df = match_spotify_data(tracks,albums,artists,audio_features,lyrics_features)
    
    train_model(df,0,features,'SpotifyRecommender_apiFeatures.pkl')
    
    


# Modell mit allen Featurs erstellen

In [4]:
create_full_model() # ca. 2h

In [62]:
model=SpotifyRecommender()
# model.load('SpotifyRecommender_allFeatures.pkl')
model.load('models/SpotifyRecommender_allFeatures.pkl')

In [8]:
track_id = '0EYOdF5FCkgOJJla8DI2Md'
data_for_prediction = model.dataset[model.dataset['track_id']==track_id]

In [63]:
output = model.predict(data_for_prediction,k=40,match_genre=True)
output.head(10)

,name,artists_name,artists_genres,album_name,track_href,preview_url,analysis_url,href,lyrics,playlist,...,artists_followers,artists_artist_popularity,artists_id,album_release_date,album_images,album_total_tracks,album_external_urls,album_id,distances,match_genre_count
700,B.Y.O.B.,System Of A Down,"[alternative metal, nu metal, post-grunge, ...",Mezmerize,https://api.spotify.com/v1/tracks/0EYOdF5FCkgO...,https://p.scdn.co/mp3-preview/2f373f029e1110b1...,https://api.spotify.com/v1/audio-analysis/0EYO...,https://api.spotify.com/v1/tracks/0EYOdF5FCkgO...,\r\n\r\nYou!\r\nWhy do they always send the po...,Metal Essentials,...,4774393,79,5eAWCfyUhZtHHtBdNk56l1,2005-05-17,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",11,{'spotify': 'https://open.spotify.com/album/0c...,0cn6MHyx4YuZauaB7Pb66o,0.000000,7
45142,Falling Apart,Papa Roach,"[alternative metal, nu metal, pop punk, pos...",F.E.A.R.,https://api.spotify.com/v1/tracks/1Ka4sX789wNf...,https://p.scdn.co/mp3-preview/52b4d4d3966ffc77...,https://api.spotify.com/v1/audio-analysis/1Ka4...,https://api.spotify.com/v1/tracks/1Ka4sX789wNf...,\r\n\r\nI'll follow you out of the dark\r\nI t...,Pegando Pesado,...,2527961,76,4RddZ3iHvSpGV4dvATac9X,2015-01-27,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",12,{'spotify': 'https://open.spotify.com/album/70...,70TYHf7Is70KOY1sHl5Vlh,7.544761,7
25517,State of Love and Trust,Pearl Jam,"[alternative rock, grunge, permanent wave, ...",rearviewmirror (greatest hits 1991-2003),https://api.spotify.com/v1/tracks/6dnmovOCP7vG...,https://p.scdn.co/mp3-preview/4c532a394fd3764b...,https://api.spotify.com/v1/audio-analysis/6dnm...,https://api.spotify.com/v1/tracks/6dnmovOCP7vG...,\r\nState of love and trust as I busted down t...,1990-luvun alternative ja grunge,...,4400511,78,1w5Kfo2jwwIPruYS2UWh56,2004-11-16,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",33,{'spotify': 'https://open.spotify.com/album/1G...,1G1R5dY01DSyti3NaWnOp3,7.677549,4
89242,One Word,Baby Animals,"[australian alternative rock, australian pop,...",Baby Animals - 25th Anniversary (Deluxe Edition),https://api.spotify.com/v1/tracks/7MKtgvzqOjyZ...,https://p.scdn.co/mp3-preview/5bc898b0759c167d...,https://api.spotify.com/v1/audio-analysis/7MKt...,https://api.spotify.com/v1/tracks/7MKtgvzqOjyZ...,\r\nOh I wake up feeling the same way every da...,90s Workout Beats,...,47330,44,2DaDoR6WXStRctDQDWWQpI,1991-05-20,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",28,{'spotify': 'https://open.spotify.com/album/2i...,2iwQHQZ2a2eyNjuQmfSdJI,8.082564,3
72880,Two Doors Down,Mystery Jets,"[alternative dance, british indie rock, danc...",Twenty One,https://api.spotify.com/v1/tracks/5JSF5Nhw9W6j...,https://p.scdn.co/mp3-preview/a017fd07956d9cc7...,https://api.spotify.com/v1/audio-analysis/5JSF...,https://api.spotify.com/v1/tracks/5JSF5Nhw9W6j...,\r\n\r\nI think I'm in love with the girl next...,Indie Workout,...,108665,51,09K1H1DgyIXHsMx2j7KTFX,2008-03-21,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",11,{'spotify': 'https://open.spotify.com/album/2e...,2efbbdHh1iSps9yfBZdy7G,8.116682,3
55117,Awakening,Throw The Fight,"[christian rock, gymcore, hard alternative, ...",Awakening,https://api.spotify.com/v1/tracks/4EmokJEqiyqI...,https://p.scdn.co/mp3-preview/3a99df675b2a2164...,https://api.spotify.com/v1/audio-analysis/4Emo...,https://api.spotify.com/v1/tracks/4EmokJEqiyqI...,\r\nI'll follow you into the dark\r\nI'll be t...,Rock Hard,...,18203,49,1nRfyHvAh1foLjOCa3hbv7,2019-01-11,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",1,{'spotify': 'https://open.spotify.com/album/56...,563WVcixzFSli4vAEm3Qxr,8.164526,3
45131,Drowned in Gold,Boston Manor,"[alternative emo, anthem emo, uk pop punk]",Drowned in Gold,https://api.spotify.com/v1/tracks/4t7xcd1KMNzx...,https://p.scdn.co/mp3-preview/eed280559725cd2b...,https://api.spotify.com/v1/audio-analysis/4t7x...,https://api.spotify.com/v1/tracks/4t7xcd1KMNzx...,"\r\n\r\nChoose love, choose tears\r\nChoose bu...",Rock Save the Queen,...,51093,5

In [23]:
data_for_prediction['artists_genres'].tolist()[0]

['alternative metal', ' nu metal', ' post-grunge', ' rap metal', ' rock']

In [30]:
def match_count(row, target_list):
    return len(set(row).intersection(target_list))


output['match_count'] = output['artists_genres'].apply(match_count, target_list=data_for_prediction['artists_genres'].tolist()[0])
output.sort_values('match_count', ascending=False).head(10)


,name,artists_name,artists_genres,album_name,track_href,preview_url,analysis_url,href,lyrics,playlist,...,artists_artist_popularity,artists_id,album_release_date,album_images,album_total_tracks,album_external_urls,album_id,distances,match_count,match_substring_count
700,B.Y.O.B.,System Of A Down,"[alternative metal, nu metal, post-grunge, ...",Mezmerize,https://api.spotify.com/v1/tracks/0EYOdF5FCkgO...,https://p.scdn.co/mp3-preview/2f373f029e1110b1...,https://api.spotify.com/v1/audio-analysis/0EYO...,https://api.spotify.com/v1/tracks/0EYOdF5FCkgO...,\r\n\r\nYou!\r\nWhy do they always send the po...,Metal Essentials,...,79,5eAWCfyUhZtHHtBdNk56l1,2005-05-17,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",11,{'spotify': 'https://open.spotify.com/album/0c...,0cn6MHyx4YuZauaB7Pb66o,0.000000,5,7
45142,Falling Apart,Papa Roach,"[alternative metal, nu metal, pop punk, pos...",F.E.A.R.,https://api.spotify.com/v1/tracks/1Ka4sX789wNf...,https://p.scdn.co/mp3-preview/52b4d4d3966ffc77...,https://api.spotify.com/v1/audio-analysis/1Ka4...,https://api.spotify.com/v1/tracks/1Ka4sX789wNf...,\r\n\r\nI'll follow you out of the dark\r\nI t...,Pegando Pesado,...,76,4RddZ3iHvSpGV4dvATac9X,2015-01-27,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",12,{'spotify': 'https://open.spotify.com/album/70...,70TYHf7Is70KOY1sHl5Vlh,7.544761,4,7
25517,State of Love and Trust,Pearl Jam,"[alternative rock, grunge, permanent wave, ...",rearviewmirror (greatest hits 1991-2003),https://api.spotify.com/v1/tracks/6dnmovOCP7vG...,https://p.scdn.co/mp3-preview/4c532a394fd3764b...,https://api.spotify.com/v1/audio-analysis/6dnm...,https://api.spotify.com/v1/tracks/6dnmovOCP7vG...,\r\nState of love and trust as I busted down t...,1990-luvun alternative ja grunge,...,78,1w5Kfo2jwwIPruYS2UWh56,2004-11-16,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",33,{'spotify': 'https://open.spotify.com/album/1G...,1G1R5dY01DSyti3NaWnOp3,7.677549,2,4
30716,Headstrong,Saviour,[australian post-hardcore],Headstrong,https://api.spotify.com/v1/tracks/0exunPxmEEME...,https://p.scdn.co/mp3-preview/b92a175b7efc4437...,https://api.spotify.com/v1/audio-analysis/0exu...,https://api.spotify.com/v1/tracks/0exunPxmEEME...,\r\nYou can only be headstrong\r\nFor so long\...,Homegrown + Heavy,...,38,6xVW2xsoMQbvcUTwqYxf0y,2018-04-02,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",1,{'spotify': 'https://open.spotify.com/album/6s...,6s96bvTJTTp76rklucRK4I,7.036965,0,0
5992,Paranoia,A Day To Remember,"[metalcore, pop punk, screamo]",Bad Vibrations (Deluxe Edition),https://api.spotify.com/v1/tracks/3HhaSocKwTpt...,https://p.scdn.co/mp3-preview/69c620e40a904eec...,https://api.spotify.com/v1/audio-analysis/3Hha...,https://api.spotify.com/v1/tracks/3HhaSocKwTpt...,\r\n\r\nExpect the worst of everyone I meet\r\...,Rage Quit,...,72,4NiJW4q9ichVqL1aUsgGAN,2016-09-02,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",13,{'spotify': 'https://open.spotify.com/album/5r...,5rSgMFmmuaKQxma4bSEviU,8.141161,0,1
32683,You Are Everything,Holding Absence,"[melodic hardcore, metalcore]",Holding Absence,https://api.spotify.com/v1/tracks/6RJSpUgLZ46a...,https://p.scdn.co/mp3-preview/2c3eae5ade97857a...,https://api.spotify.com/v1/audio-analysis/6RJS...,https://api.spotify.com/v1/tracks/6RJSpUgLZ46a...,\r\n\r\nSome kind of doubt poisons my mind\r\n...,Crash Course,...,46,2tl280wIokrLjabzrSKTgU,2019-03-01,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",11,{'spotify': 'https://open.spotify.com/album/6x...,6xEQWs9XbaJBGAy4Jv7piM,8.147830,0,1
55348,Now / Never,She Pulled the Trigger,"[gymcore, post-screamo]",No Matter What,https://api.spotify.com/v1/tracks/1962zHkiWIAV...,https://p.scdn.co/mp3-preview/cd282bf9d64fb6c9...,https://api.spotify.com/v1/audio-analysis/1962...,https://api.spotify.com/v1/tracks/1962zHkiWIAV...,"\r\n\r\nHave you heard the news, my friend?\r\...",Crash Course,...,25,6u8zncAVE7R0ezzTOrBIZt,2018-06-22,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",12,{'spotify': 'https://open.spotify.co

In [31]:
def match_substring_count(row, target_list):
    target_words = set(word for genre in target_list for word in genre.split(' '))
    row_words = set(word for genre in row for word in genre.split(' '))
    return len(row_words.intersection(target_words))

output['match_substring_count'] = output['artists_genres'].apply(match_substring_count, target_list=data_for_prediction['artists_genres'].tolist()[0])


output = output.sort_values(by=['match_substring_count','distances'], ascending=[False,True])
output.head(10)



,name,artists_name,artists_genres,album_name,track_href,preview_url,analysis_url,href,lyrics,playlist,...,artists_artist_popularity,artists_id,album_release_date,album_images,album_total_tracks,album_external_urls,album_id,distances,match_count,match_substring_count
700,B.Y.O.B.,System Of A Down,"[alternative metal, nu metal, post-grunge, ...",Mezmerize,https://api.spotify.com/v1/tracks/0EYOdF5FCkgO...,https://p.scdn.co/mp3-preview/2f373f029e1110b1...,https://api.spotify.com/v1/audio-analysis/0EYO...,https://api.spotify.com/v1/tracks/0EYOdF5FCkgO...,\r\n\r\nYou!\r\nWhy do they always send the po...,Metal Essentials,...,79,5eAWCfyUhZtHHtBdNk56l1,2005-05-17,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",11,{'spotify': 'https://open.spotify.com/album/0c...,0cn6MHyx4YuZauaB7Pb66o,0.000000,5,7
45142,Falling Apart,Papa Roach,"[alternative metal, nu metal, pop punk, pos...",F.E.A.R.,https://api.spotify.com/v1/tracks/1Ka4sX789wNf...,https://p.scdn.co/mp3-preview/52b4d4d3966ffc77...,https://api.spotify.com/v1/audio-analysis/1Ka4...,https://api.spotify.com/v1/tracks/1Ka4sX789wNf...,\r\n\r\nI'll follow you out of the dark\r\nI t...,Pegando Pesado,...,76,4RddZ3iHvSpGV4dvATac9X,2015-01-27,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",12,{'spotify': 'https://open.spotify.com/album/70...,70TYHf7Is70KOY1sHl5Vlh,7.544761,4,7
25517,State of Love and Trust,Pearl Jam,"[alternative rock, grunge, permanent wave, ...",rearviewmirror (greatest hits 1991-2003),https://api.spotify.com/v1/tracks/6dnmovOCP7vG...,https://p.scdn.co/mp3-preview/4c532a394fd3764b...,https://api.spotify.com/v1/audio-analysis/6dnm...,https://api.spotify.com/v1/tracks/6dnmovOCP7vG...,\r\nState of love and trust as I busted down t...,1990-luvun alternative ja grunge,...,78,1w5Kfo2jwwIPruYS2UWh56,2004-11-16,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",33,{'spotify': 'https://open.spotify.com/album/1G...,1G1R5dY01DSyti3NaWnOp3,7.677549,2,4
89242,One Word,Baby Animals,"[australian alternative rock, australian pop,...",Baby Animals - 25th Anniversary (Deluxe Edition),https://api.spotify.com/v1/tracks/7MKtgvzqOjyZ...,https://p.scdn.co/mp3-preview/5bc898b0759c167d...,https://api.spotify.com/v1/audio-analysis/7MKt...,https://api.spotify.com/v1/tracks/7MKtgvzqOjyZ...,\r\nOh I wake up feeling the same way every da...,90s Workout Beats,...,44,2DaDoR6WXStRctDQDWWQpI,1991-05-20,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",28,{'spotify': 'https://open.spotify.com/album/2i...,2iwQHQZ2a2eyNjuQmfSdJI,8.082564,0,3
72880,Two Doors Down,Mystery Jets,"[alternative dance, british indie rock, danc...",Twenty One,https://api.spotify.com/v1/tracks/5JSF5Nhw9W6j...,https://p.scdn.co/mp3-preview/a017fd07956d9cc7...,https://api.spotify.com/v1/audio-analysis/5JSF...,https://api.spotify.com/v1/tracks/5JSF5Nhw9W6j...,\r\n\r\nI think I'm in love with the girl next...,Indie Workout,...,51,09K1H1DgyIXHsMx2j7KTFX,2008-03-21,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",11,{'spotify': 'https://open.spotify.com/album/2e...,2efbbdHh1iSps9yfBZdy7G,8.116682,0,3
55117,Awakening,Throw The Fight,"[christian rock, gymcore, hard alternative, ...",Awakening,https://api.spotify.com/v1/tracks/4EmokJEqiyqI...,https://p.scdn.co/mp3-preview/3a99df675b2a2164...,https://api.spotify.com/v1/audio-analysis/4Emo...,https://api.spotify.com/v1/tracks/4EmokJEqiyqI...,\r\nI'll follow you into the dark\r\nI'll be t...,Rock Hard,...,49,1nRfyHvAh1foLjOCa3hbv7,2019-01-11,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",1,{'spotify': 'https://open.spotify.com/album/56...,563WVcixzFSli4vAEm3Qxr,8.164526,0,3
45131,Drowned in Gold,Boston Manor,"[alternative emo, anthem emo, uk pop punk]",Drowned in Gold,https://api.spotify.com/v1/tracks/4t7xcd1KMNzx...,https://p.scdn.co/mp3-preview/eed280559725cd2b...,https://api.spotify.com/v1/audio-analysis/4t7x...,https://api.spotify.com/v1/tracks/4t7xcd1KMNzx...,"\r\n\r\nChoose love, choose tears\r\nChoose bu...",Rock Save the Queen,...,52,4WjeQi9wm84lYTIWZ95QoM,2017-12-05,"[{

# Modell mit den Haupt-Features 

apiFeature sind nur features, die aus der Spotify API als audio_features abgerufen werden können

In [9]:
create_apiFeatures_model() # ca 6 min

In [57]:
model=SpotifyRecommender()
model.load('SpotifyRecommender_apiFeatures.pkl')
# model.load('models/SpotifyRecommender_apiFeatures.pkl')

In [58]:
track_id = '0EYOdF5FCkgOJJla8DI2Md'
data_for_prediction = model.dataset[model.dataset['track_id']==track_id]
data_for_prediction[model.return_columns]

,name,artists_name,artists_genres,album_name,track_href,preview_url,analysis_url,href,lyrics,playlist,...,time_signature,track_id,artists_followers,artists_artist_popularity,artists_id,album_release_date,album_images,album_total_tracks,album_external_urls,album_id
700,B.Y.O.B.,System Of A Down,"[alternative metal, nu metal, post-grunge, ...",Mezmerize,https://api.spotify.com/v1/tracks/0EYOdF5FCkgO...,https://p.scdn.co/mp3-preview/2f373f029e1110b1...,https://api.spotify.com/v1/audio-analysis/0EYO...,https://api.spotify.com/v1/tracks/0EYOdF5FCkgO...,\r\n\r\nYou!\r\nWhy do they always send the po...,Metal Essentials,...,4.0,0EYOdF5FCkgOJJla8DI2Md,4774393,79,5eAWCfyUhZtHHtBdNk56l1,2005-05-17,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",11,{'spotify': 'https://open.spotify.com/album/0c...,0cn6MHyx4YuZauaB7Pb66o


In [61]:
output = model.predict(data_for_prediction,k=40,match_genre=True)
output.head(10)

,name,artists_name,artists_genres,album_name,track_href,preview_url,analysis_url,href,lyrics,playlist,...,artists_followers,artists_artist_popularity,artists_id,album_release_date,album_images,album_total_tracks,album_external_urls,album_id,distances,match_genre_count
700,B.Y.O.B.,System Of A Down,"[alternative metal, nu metal, post-grunge, ...",Mezmerize,https://api.spotify.com/v1/tracks/0EYOdF5FCkgO...,https://p.scdn.co/mp3-preview/2f373f029e1110b1...,https://api.spotify.com/v1/audio-analysis/0EYO...,https://api.spotify.com/v1/tracks/0EYOdF5FCkgO...,\r\n\r\nYou!\r\nWhy do they always send the po...,Metal Essentials,...,4774393,79,5eAWCfyUhZtHHtBdNk56l1,2005-05-17,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",11,{'spotify': 'https://open.spotify.com/album/0c...,0cn6MHyx4YuZauaB7Pb66o,9.964058e-15,7
43778,Happy?,Mudvayne,"[alternative metal, groove metal, industrial...",Lost and Found,https://api.spotify.com/v1/tracks/0INUE1K7cEiE...,https://p.scdn.co/mp3-preview/a4972554181a20b1...,https://api.spotify.com/v1/audio-analysis/0INU...,https://api.spotify.com/v1/tracks/0INUE1K7cEiE...,\r\nIn this hole\r\nThat is me\r\nThe dead are...,Metal Essentials,...,700048,62,2Pfv2w8a20xzC7Dr7QXRqM,2005-01-01,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",12,{'spotify': 'https://open.spotify.com/album/0d...,0dCRu5tVhx9fvbVZSxfjfP,4.152681e-01,7
722,Be Legendary,Pop Evil,"[alternative metal, nu metal, post-grunge, ...",Pop Evil,https://api.spotify.com/v1/tracks/2mVFo8BJ6ZPh...,https://p.scdn.co/mp3-preview/f430fee71dd2ef47...,https://api.spotify.com/v1/audio-analysis/2mVF...,https://api.spotify.com/v1/tracks/2mVFo8BJ6ZPh...,"\r\nPull the trigger, ride the bull\r\nAnd cut...",Rock This,...,270542,63,1pRaG81GsVtaTBuVSpldt2,2018-02-16,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",11,{'spotify': 'https://open.spotify.com/album/2z...,2zrUlKeTyEhyYulj8Gu8T9,5.672768e-01,7
26161,No More,Disturbed,"[alternative metal, nu metal, post-grunge, ...",Evolution (Deluxe),https://api.spotify.com/v1/tracks/2xECSFHPgCyE...,https://p.scdn.co/mp3-preview/40055a0e9aef53d4...,https://api.spotify.com/v1/audio-analysis/2xEC...,https://api.spotify.com/v1/tracks/2xECSFHPgCyE...,\r\n\r\nOnce again we hear them calling for wa...,Heviä treeneihin,...,2771476,78,3TOqt5oJwL9BE2NG9MEwDa,2018-10-19,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",14,{'spotify': 'https://open.spotify.com/album/1I...,1IpufqmucrxYUJQxf1AP0n,1.973208e-01,6
26150,Open Your Eyes,Disturbed,"[alternative metal, nu metal, post-grunge, ...",Immortalized,https://api.spotify.com/v1/tracks/34bimdZj8GMH...,https://p.scdn.co/mp3-preview/7200198474bd64db...,https://api.spotify.com/v1/audio-analysis/34bi...,https://api.spotify.com/v1/tracks/34bimdZj8GMH...,\r\n\r\nYou're paralyzed\r\nBreathe your disgu...,Rock til arbejdet,...,2771476,78,3TOqt5oJwL9BE2NG9MEwDa,2015-08-21,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",13,{'spotify': 'https://open.spotify.com/album/3q...,3qFQ4XNQ15alZrAaj5oGJK,5.486804e-01,6
26166,Land Of Confusion,Disturbed,"[alternative metal, nu metal, post-grunge, ...",Ten Thousand Fists (Standard Edition),https://api.spotify.com/v1/tracks/4ClGNWLK9vZM...,https://p.scdn.co/mp3-preview/d829a0f99289b0e2...,https://api.spotify.com/v1/audio-analysis/4ClG...,https://api.spotify.com/v1/tracks/4ClGNWLK9vZM...,\r\n\r\nI must have dreamed a thousand dreams\...,Rock Covers,...,2771476,78,3TOqt5oJwL9BE2NG9MEwDa,2005-09-19,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",14,{'spotify': 'https://open.spotify.com/album/0T...,0Te7OvzuUMnbsqCneIDUm6,5.568184e-01,6
17079,All My Friends are Nobodies,zebrahead,"[funk metal, nu metal, pop punk, punk, rap...",Brain Invaders,https://api.spotify.com/v1/tracks/2iFFcHvg5F7a...,https://p.scdn.co/mp3-preview/9bdb08f7d16e27fc...,https://api.spotify.com/v1/audio-analysis/2iFF...,https://api.spotify.com/v1/tracks/2iFFcHvg5F7a...,\r\n\r\nTo all my friends who are just like me...,Punk Rock Workout,...,141967,58,6SiyKSeJo6gcsS2NvuAbsl,2019-03-06